<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/bert-base-paradetox-editOps-preprocess-v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.6 MB/s eta 0:00:00


In [2]:
from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader

In [3]:
# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define special tokens and add them to the tokenizer
special_tokens_dict = {'additional_special_tokens': ['[INS]', '[DEL]']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

In [4]:
# Define dataset preprocessing
def preprocess_function(examples):
    # Tokenize the masked comments
    result = tokenizer(examples['masked_comment'], truncation=True, padding='max_length', max_length=128)

    # We need to tell the model to ignore the loss for non-masked tokens
    # BERT uses -100 to ignore loss calculation for a token
    result["labels"] = [list(map(lambda x: -100 if x == tokenizer.pad_token_id else x, seq)) for seq in result["input_ids"]]

    return result

In [5]:
dataset = load_dataset("HamdanXI/paradetox_editOps_preprocess")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/19744 [00:00<?, ? examples/s]

In [6]:
# Preprocess and tokenize the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Load the pre-trained model
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.resize_token_embeddings(len(tokenizer))  # Resize model embedding to fit new tokens

Map:   0%|          | 0/19744 [00:00<?, ? examples/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Embedding(30524, 768)

In [14]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='bert-base-paradetox-editOps-preprocess',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_strategy="epoch",
    logging_strategy="epoch",
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

# Train the model
trainer.train()

Step,Training Loss
2468,0.000700
4936,0.000100
7404,0.000000


TrainOutput(global_step=7404, training_loss=0.0002890472174061111, metrics={'train_runtime': 637.8704, 'train_samples_per_second': 92.859, 'train_steps_per_second': 11.607, 'total_flos': 3897536958922752.0, 'train_loss': 0.0002890472174061111, 'epoch': 3.0})

In [8]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [9]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# model.save_pretrained('bert-base-paradetox-editOps-preprocess')
tokenizer.save_pretrained('bert-base-paradetox-editOps-preprocess')

# model.push_to_hub('bert-base-paradetox-editOps-preprocess')
tokenizer.push_to_hub('bert-base-paradetox-editOps-preprocess')

In [15]:
trainer.push_to_hub()

events.out.tfevents.1699258981.ed7ffc97f7a7.283.2:   0%|          | 0.00/5.12k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

events.out.tfevents.1699258960.ed7ffc97f7a7.283.1:   0%|          | 0.00/4.29k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

'https://huggingface.co/HamdanXI/bert-base-paradetox-editOps-preprocess/tree/main/'

In [17]:
from transformers import BertTokenizer, BertForMaskedLM
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define special tokens for insertion and deletion
tokenizer.add_tokens(['[INS]', '[DEL]'])

# Assume we have a special token <del> to represent deletions in the model
del_token = tokenizer.encode('<del>', add_special_tokens=False)[0]
tokenizer.add_special_tokens({'additional_special_tokens': ['<del>']})

# Assume we have a dataset in the format:
# dataset['train'][0] = {
#    'masked_comment': 'he [MASK] [INS] brave [INS] too!',
#    'edit_ops': [['replace', 'had steel balls too !', '1', 'was brave too!']]
# }

# Preprocess the dataset to replace [INS] with [MASK] and [DEL] with the word to delete
def preprocess_function(examples):
    inputs = []
    labels = []

    for masked_comment in examples['masked_comment']:
        # Replace [INS] with [MASK] and create labels for inserted words
        # Replace [DEL] with the actual word and set label to <del> token
        masked_input = masked_comment
        masked_labels = masked_comment
        masked_input = masked_input.replace('[INS]', tokenizer.mask_token).replace('[DEL]', '[DEL]')  # Replace with [DEL] temporarily
        masked_labels = masked_labels.replace('[INS]', '[INS]').replace('[DEL]', '<del>')  # Replace with <del> token

        # Tokenize the inputs and labels
        inputs.append(tokenizer(masked_input, padding='max_length', truncation=True, max_length=128))
        labels.append(tokenizer(masked_labels, padding='max_length', truncation=True, max_length=128))

    # Map tokens to their IDs for inputs and labels
    input_ids = [x['input_ids'] for x in inputs]
    label_ids = [x['input_ids'] for x in labels]  # note that we use input_ids here because that's our labels

    # Create the final labels by setting non-[MASK] tokens to -100 so they are not considered in the loss
    final_labels = []
    for i, label in enumerate(label_ids):
        label_row = []
        for j, token_id in enumerate(label):
            # If the corresponding input token is not a mask token, we set the label to -100
            if input_ids[i][j] != tokenizer.mask_token_id:
                label_row.append(-100)
            else:
                label_row.append(token_id)
        final_labels.append(label_row)

    return {'input_ids': input_ids, 'labels': final_labels, 'attention_mask': [x['attention_mask'] for x in inputs]}

# Load your dataset here
dataset = load_dataset("HamdanXI/paradetox_editOps_preprocess")

# Apply the preprocessing function to the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Load the model and resize token embeddings
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.resize_token_embeddings(len(tokenizer))

# Training arguments
training_args = TrainingArguments(
    output_dir="bert-base-paradetox-editOps-preprocess-v2",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_strategy="epoch",
    logging_strategy="epoch",
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

# Train the model
trainer.train()

Parameter 'function'=<function preprocess_function at 0x79cab4d7aa70> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/19744 [00:00<?, ? examples/s]

KeyboardInterrupt: ignored